In [1]:
import numpy as np
import math
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

In [2]:
from torch import nn
from torch.nn import functional as F

In [3]:
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.1307,),(0.3081,)), # mean value = 0.1307, standard deviation value = 0.3081
])

In [6]:
data_path = './MNIST'

training_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)

In [8]:
print("the number of your training data (must be 10,000) = ", training_set.__len__()) 
print("the number of your testing data (must be 60,000) = ", testing_set.__len__())

the number of your training data (must be 10,000) =  10000
the number of your testing data (must be 60,000) =  60000


In [10]:
class MyModel(nn.Module):

    def __init__(self, num_classes=10, size_kernel=5):

        super(MyModel, self).__init__()

        # *********************************************************************
        # input parameter
        #
        # data size:
        #   mnist   : 28 * 28
        # *********************************************************************
        self.number_class   = num_classes
        self.size_kernel    = size_kernel        
        
        # *********************************************************************
        # feature layer
        # *********************************************************************
        self.conv1          = nn.Conv2d(1, 20, kernel_size=size_kernel, stride=1, padding=int((size_kernel-1)/2), bias=True)
        self.conv2          = nn.Conv2d(20, 50, kernel_size=size_kernel, stride=1, padding=int((size_kernel-1)/2), bias=True)

        self.conv_layer1    = nn.Sequential(self.conv1, nn.MaxPool2d(kernel_size=2), nn.ReLU(True))
        self.conv_layer2    = nn.Sequential(self.conv2, nn.MaxPool2d(kernel_size=2), nn.ReLU(True))

        self.feature        = nn.Sequential(self.conv_layer1, self.conv_layer2)
        
        # *********************************************************************
        # classifier layer
        # *********************************************************************
        self.fc1        = nn.Linear(50*7*7, 50, bias=True)
        self.fc2        = nn.Linear(50, num_classes, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.ReLU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.ReLU(True))

        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2)
        
        self._initialize_weight()        
        
    def _initialize_weight(self):

        for m in self.modules():
            
            if isinstance(m, nn.Conv2d):
                
                nn.init.xavier_uniform_(m.weight, gain=math.sqrt(2))

                if m.bias is not None:

                    m.bias.data.zero_()

            elif isinstance(m, nn.Linear):

                nn.init.xavier_uniform_(m.weight, gain=math.sqrt(2))
                
                if m.bias is not None:

                    m.bias.data.zero_()

    def forward(self, x):

        x = self.feature(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        # x = F.softmax(x, dim=1)
        return x

In [11]:
batch_size = 32
l_train = torch.utils.data.DataLoader(dataset=training_set, batch_size = batch_size, shuffle=True, drop_last=True)
l_test = torch.utils.data.DataLoader(dataset=testing_set, batch_size = batch_size, shuffle=True, drop_last=True)

In [12]:
device = torch.device('cpu')
criterion = nn.CrossEntropyLoss() #change criterion
classifier = MyModel().to(device)
alph = 1e-3 #change learning rate
optimizer = torch.optim.Adam(classifier.parameters(), lr=alph) #change optimizer
training_epochs = 80 #change epoch

In [13]:
tr_Liters = []; tr_Aiters = []
te_Liters = []; te_Aiters = []
tr_size = len(l_train); te_size = len(l_test)

In [ ]:
for epoch in range(training_epochs):
    tr_avg_loss=0
    tr_avg_acc=0
    te_avg_loss=0
    te_avg_acc=0
    
    if epoch % 10 == 0:
        print("epoch: ", epoch)
        
    for x, y in l_train:
        optimizer.zero_grad()
        y_pred = classifier(x)
        tr_loss = criterion(y_pred, y)
        tr_loss.backward()
        optimizer.step()
        
        tr_avg_loss += tr_loss.item() / tr_size
        fin_pred = torch.argmax(y_pred, 1) == y
        tr_acc = fin_pred.float().mean()
        tr_avg_acc += tr_acc.item() / tr_size
        
        del tr_loss
        del y_pred #이거 지울 준비
        
    with torch.no_grad(): #이거 지울 준비
        for x, y in l_test:
            y_pred = classifier(x)
            te_loss = criterion(y_pred,y)
            
            te_avg_loss += te_loss.item() / te_size
            fin_pred = torch.argmax(y_pred, 1) == y
            te_acc = fin_pred.float().mean()
            te_avg_acc += te_acc.item() / te_size
            
            del te_loss
            del y_pred
            
    tr_Liters.append(tr_avg_loss); tr_Aiters.append(tr_avg_acc)
    te_Liters.append(te_avg_loss); te_Aiters.append(te_avg_acc)

epoch:  0
epoch:  10
epoch:  20
epoch:  30


In [ ]:
#바꾸자
loss_df = pd.DataFrame(data=np.array([[tr_Liters[-1]], [te_Liters[-1]]]), index=['training', 'testing'], columns=[' '])
loss_df.columns.name = "loss"
loss_df[' '] = loss_df[' '].map('{:,.5f}'.format)

In [ ]:
accuracy_df = pd.DataFrame(data=np.array([[round(tr_Aiters[-1], 5)], [round(te_Aiters[-1], 5)]]),
index=['training', 'testing'],
columns=[' '])
accuracy_df.columns.name = "accuracy"
accuracy_df[' '] = accuracy_df[' '].map('{:,.5f}'.format)

In [ ]:
plt.figure(1, figsize=(8, 8))
plt.plot(np.array(range(training_epochs)), tr_Liters, c='red', label='train loss') 
plt.plot(np.array(range(training_epochs)), te_Liters, c='blue', label='test loss') plt.title('loss')
plt.xticks(range(0, training_epochs, 10))
plt.legend()
plt.show()

In [ ]:
loss_df

In [ ]:
accuracy_df